In [1]:
#Импортируем нужные модули
import numpy as np 
import pandas as pd
import sys, re, csv, codecs

import matplotlib.pyplot as plt
%matplotlib inline

import os
#!pip install tensorflow
#!pip install keras
import keras
from keras.models import Sequential
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU, CuDNNLSTM, CuDNNGRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import GlobalAveragePooling1D, GlobalMaxPool1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from keras import backend as K
from keras.engine.topology import Layer, InputSpec

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('updated_test.csv')

In [3]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
y_te = test[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [4]:
df_combined =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

print(df_combined.shape)

(223549, 8)


Теперь создадим словарь всех уникальных слов в нашей выборке с помощью встроенного в Keras класса Tokenizer, он просто создаёт словарь с уникальными токенами и сортирует их по количеству употреблений в наших комментариях.

Затем сформируем векторы фиксированной длины: слишком короткие вектора заполнятся нулями в конце (padding = 'post', а слишком длинные будут обрезаны(truncating = 'post' с конца) до требуемой длины, эта длина равна 150.

In [5]:
docs_combined = df_combined['comment_text'].astype(str)
t = Tokenizer()
t.fit_on_texts(docs_combined)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs_combined)
padded_docs_combined = pad_sequences(encoded_docs, maxlen=150, padding='post')

In [6]:
df_train_padded = padded_docs_combined[:len(train)]
df_test_padded = padded_docs_combined[len(train):]

print(df_train_padded.shape)
print(df_test_padded.shape)

(159571, 150)
(63978, 150)


In [7]:
X_t = df_train_padded
X_te = df_test_padded

In [54]:
t.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'you': 6,
 'i': 7,
 'is': 8,
 'that': 9,
 'in': 10,
 'it': 11,
 '\r': 12,
 'for': 13,
 'this': 14,
 'not': 15,
 'on': 16,
 'be': 17,
 'as': 18,
 'are': 19,
 'have': 20,
 'your': 21,
 'with': 22,
 'if': 23,
 'article': 24,
 'was': 25,
 'or': 26,
 'but': 27,
 'page': 28,
 'an': 29,
 'wikipedia': 30,
 'my': 31,
 'from': 32,
 'by': 33,
 'at': 34,
 'do': 35,
 'about': 36,
 'so': 37,
 'me': 38,
 'what': 39,
 'can': 40,
 'there': 41,
 'talk': 42,
 'all': 43,
 'has': 44,
 'will': 45,
 'would': 46,
 'no': 47,
 'please': 48,
 'one': 49,
 'like': 50,
 'just': 51,
 'they': 52,
 'he': 53,
 'which': 54,
 'any': 55,
 'been': 56,
 'we': 57,
 'should': 58,
 'more': 59,
 'some': 60,
 "don't": 61,
 'other': 62,
 'who': 63,
 'see': 64,
 'here': 65,
 'think': 66,
 'also': 67,
 'his': 68,
 'know': 69,
 'because': 70,
 'how': 71,
 "i'm": 72,
 "it's": 73,
 'up': 74,
 'people': 75,
 'edit': 76,
 'only': 77,
 'why': 78,
 'out': 79,
 'am': 80,
 'when': 81,
 'us

In [55]:
t.word_counts

OrderedDict([('explanation', 2433),
             ('why', 23771),
             ('the', 684748),
             ('edits', 12776),
             ('made', 13036),
             ('under', 9380),
             ('my', 59258),
             ('username', 2454),
             ('hardcore', 224),
             ('metallica', 51),
             ('fan', 1284),
             ('were', 21160),
             ('reverted', 5100),
             ('they', 37879),
             ("weren't", 596),
             ('vandalisms', 44),
             ('just', 38339),
             ('closure', 131),
             ('on', 121775),
             ('some', 31280),
             ('gas', 483),
             ('after', 11452),
             ('i', 273702),
             ('voted', 432),
             ('at', 54330),
             ('new', 14681),
             ('york', 1247),
             ('dolls', 30),
             ('fac', 509),
             ('and', 305403),
             ('please', 39856),
             ("don't", 31262),
             ('remove', 6900),
    

In [56]:
len(t.word_index)

300258

In [58]:
X_t[0,]

array([  691,    78,     1,   134,   131,   180,    31,   682,  4600,
       11955,  1196,    86,   352,    52,  2251, 13034,    51,  6664,
          16,    60,  2652,   149,     7,  2892,    34,   117,  1231,
       16474,  2535,     4,    48,    61,   249,     1,   364,    32,
           1,    42,    28,   144,    72,  3623,    90,  3663,  5461,
        2457,  1095,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [59]:
X_t.shape

(159571, 150)

In [60]:
y.shape

(159571, 6)

Затем мы передаем его в слой Embedding'а, где мы проецируем слова в определенное векторное пространство в зависимости от контекста, "расстояния" до близлежащих слов. Мы будем использовать модель векторного представления слов GloVe, которая позволит нам учесть семантические особенности слов, в отличие от того же подхода Bag of Words. Нам также удастся сократить длину вектора слова.

Заготавливаем матрицу для векторного представления GloVe(будем использовать модель с размерностью 300 для векторов-слов, предобученную на корпусе данных Common crawl) - подробнее о GloVe: https://nlp.stanford.edu/projects/glove/ 

In [10]:
#словарь типа слово - вектор GloVe
embeddings_index = dict()
f = open('glove.840B.300d.txt', encoding="utf8")

for line in f:
    # Note: use split(' ') instead of split() if you get an error.
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Всего загрузили %s векторов слов' % len(embeddings_index))

#матрица эмбеддингов
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Всего загрузили 2196017 векторов слов


## Построение сетки с архитектурой BiLSTM 

Разобьём трэйн для валидации модели.

In [11]:
X_train, X_eval, y_train ,y_eval = train_test_split(X_t, y, test_size=0.2, shuffle=True)

In [12]:
embeddings_index['mathematics']

array([ 1.9227e-01,  3.3137e-01,  3.3429e-01, -2.2540e-01, -8.3662e-02,
        1.0478e-01, -1.8902e-01,  3.3231e-01,  1.4670e-01,  1.7217e+00,
       -4.2964e-02,  3.3987e-01,  2.2034e-01,  2.1953e-01,  2.1325e-01,
        4.1103e-01,  2.0759e-01,  1.1697e+00, -1.5098e-01, -3.1001e-01,
        5.3334e-01, -3.9837e-01, -1.2726e-01,  3.5655e-01,  6.1997e-01,
       -4.2241e-01,  4.2457e-01,  3.0292e-01,  2.9472e-01,  4.5940e-01,
       -2.7553e-01,  4.4170e-01,  4.3526e-01, -3.5728e-01,  1.7547e-01,
        6.3202e-02,  3.5854e-01,  2.8797e-01, -2.1634e-01, -1.5807e-01,
        4.2994e-01, -6.9031e-01, -3.5721e-01,  1.3061e-01,  1.5789e-01,
        2.8819e-01,  1.3098e-01,  2.6952e-02,  2.4545e-02, -8.7156e-02,
       -3.1600e-02,  3.6954e-01,  6.3701e-02,  3.4070e-01, -3.5219e-01,
        4.0002e-01,  3.8386e-01, -1.5852e-01, -3.5437e-01,  2.7059e-01,
       -1.6206e-01,  1.1076e-01,  3.8801e-01, -4.5731e-01,  3.2759e-01,
        3.5994e-01, -6.1825e-01,  5.6974e-01,  6.6433e-01, -4.53

In [13]:
embedding_matrix.shape

(300259, 300)

### Необходимо написать слой Attention, т.к. в keras не реализована модель "Внимания". Код слоя позаимствован у исследователей проекта DeepMogi: media.mit.edu/projects/deepmoji/overview/

In [14]:
CONTEXT_DIM = 100 #Размерность контекстного вектора = 100, выходом слоя Attention также будет вектор размерности 100

class Attention(Layer):

    def __init__(self, regularizer=regularizers.l2(1e-10), **kwargs):
        self.regularizer = regularizer
        self.supports_masking = True
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3        
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], CONTEXT_DIM),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)
        self.b = self.add_weight(name='b',
                                 shape=(CONTEXT_DIM,),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)
        self.u = self.add_weight(name='u',
                                 shape=(CONTEXT_DIM,),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)        
        super(Attention, self).build(input_shape)

    @staticmethod
    def softmax(x, dim):
        """Computes softmax along a specified dim. Keras currently lacks this feature.
        """
        if K.backend() == 'tensorflow':
            import tensorflow as tf
            return tf.nn.softmax(x, dim)
        elif K.backend() == 'theano':
            # Theano cannot softmax along an arbitrary dim.
            # So, we will shuffle `dim` to -1 and un-shuffle after softmax.
            perm = np.arange(K.ndim(x))
            perm[dim], perm[-1] = perm[-1], perm[dim]
            x_perm = K.permute_dimensions(x, perm)
            output = K.softmax(x_perm)

            # Permute back
            perm[dim], perm[-1] = perm[-1], perm[dim]
            output = K.permute_dimensions(x, output)
            return output
        else:
            raise ValueError("Backend '{}' not supported".format(K.backend()))

    def call(self, x, mask=None):
        ut = K.tanh(K.bias_add(K.dot(x, self.W), self.b)) * self.u

        # Collapse `attention_dims` to 1. This indicates the weight for each time_step.
        ut = K.sum(ut, axis=-1, keepdims=True)

        # Convert those weights into a distribution but along time axis.
        # i.e., sum of alphas along `time_steps` axis should be 1.
        self.at = self.softmax(ut, dim=1)
        if mask is not None:
            self.at *= K.cast(K.expand_dims(mask, -1), K.floatx())

        # Weighted sum along `time_steps` axis.
        return K.sum(x * self.at, axis=-2)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def get_config(self):
        config = {}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_mask(self, inputs, mask):
        return None

### Получаем, наконец, слой Embedding и строим сетку с архитектурой:
1 слой: входной - лист из комментариев, закодированных векторами длины 150 - наши pad_to_sequences

2 слой: эмбеддинги - матрица векторов слов длины 300 для каждого комментария

3 слой: spacial-дропаут для регуляризации

3 слой: эмбеддинги "скармливаются" LSTM, разверность скрытого состояния - 50. Соответственно, пройдя в двух направлениях на выходе получим канкатенацию - векторы размерности 100, т.к. используем двунаправленную LSTM.

4 слой: слой Attention - на выходе вектор размерности 100 для каждого комментария

5 слой: полносвязный с 70 нейронами - снижаем размерность до 70, активационная функция - 'ReLU'

6 слой: дроп-аут для предотвращения переобучения

7 слой: полносвязный с 6 нейронами - получаем искомые вероятности принадлежности к классам, активационная функция - сигмоидальная

In [15]:
model=Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=150, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add((Bidirectional(LSTM(50,return_sequences=True))))
model.add(Attention())
model.add(Dense(70, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation="sigmoid"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Обучаем модель и проводим валидацию:

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          90077700  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 100)          140400    
_________________________________________________________________
attention_1 (Attention)      (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 70)                7070      
_________________________________________________________________
dropout_1 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 426       
Total para

Конфигурируем быстрый выход при валидации, при этом сохраняем веса моделей

In [72]:
Adam_opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000015)
model.compile(optimizer=Adam_opt, loss='binary_crossentropy', metrics=['acc'])
#ищем пересеение кривых val_loss'a c валидационной подвыборки и loss'а с трэйна
early_stopping = EarlyStopping(monitor='val_loss', patience=4, mode='min',min_delta=0.0005)
save_best = ModelCheckpoint('lstmattention.hdf', save_best_only=True, monitor='val_acc', mode='max')

In [ ]:
history = model.fit(X_t, y, validation_data=(X_eval, y_eval), epochs=40, verbose=1,
                    callbacks=[early_stopping,save_best],batch_size=128)

Train on 159571 samples, validate on 31915 samples
Epoch 1/40
159571/159571 [==============================] - 285s 2ms/step - loss: 0.0681 - acc: 0.9778 - val_loss: 0.0436 - val_acc: 0.9832
Epoch 2/40
159571/159571 [==============================] - 281s 2ms/step - loss: 0.0454 - acc: 0.9827 - val_loss: 0.0417 - val_acc: 0.9834
Epoch 3/40
159571/159571 [==============================] - 280s 2ms/step - loss: 0.0427 - acc: 0.9835 - val_loss: 0.0388 - val_acc: 0.9845
Epoch 4/40
159571/159571 [==============================] - 280s 2ms/step - loss: 0.0411 - acc: 0.9840 - val_loss: 0.0369 - val_acc: 0.9853
Epoch 5/40
159571/159571 [==============================] - 280s 2ms/step - loss: 0.0395 - acc: 0.9845 - val_loss: 0.0352 - val_acc: 0.9858
Epoch 6/40
159571/159571 [==============================] - 280s 2ms/step - loss: 0.0384 - acc: 0.9850 - val_loss: 0.0334 - val_acc: 0.9864
Epoch 7/40
159571/159571 [==============================] - 281s 2ms/step - loss: 0.0371 - acc: 0.9853 - val_

## Предикт

In [17]:
model.load_weights('lstmattention.hdf')

In [18]:
predictions = model.predict(X_te)

In [19]:
print ("ROC-AUC: %0.5f " % roc_auc_score(y_te, predictions))

ROC-AUC: 0.98025 
